In [ ]:
import sys
print("Python executable:", sys.executable)

# Install packages into the active kernel environment
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install sentence-transformers faiss-cpu pypdf requests numpy tqdm
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter notebook

Python executable: D:\1iNstallation-Files\envs\ragbook\python.exe
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 8.5 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 4.3 MB/s  0:00:00

   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]



usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run script server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [1]:
from pypdf import PdfReader

pdf_path = r"D:\BnF Tech\RAG\FAQ Updated.pdf"  # full path to the PDF file
reader = PdfReader(pdf_path)

pages = []
for i, page in enumerate(reader.pages, start=1):
    txt = page.extract_text()
    pages.append({
        "page": i,
        "text": txt if txt else ""
    })

#print(f"Total pages read: {len(pages)}")
#for page in pages:
    #print(f"\n--- Page {page['page']} ---\n")
    #print(page["text"].replace("\n", " "))

# Robust Q&A chunker for FAQ PDF where answers may span pages.
import re, uuid
from pprint import pprint

def build_full_text_with_page_markers(pages):
    # pages is list of {"page": i, "text": ...}
    full = []
    for p in pages:
        marker = f"\n<<PAGE_{p['page']}>>\n"
        full.append(marker + (p['text'] or ""))
    return "".join(full)

def qa_chunks_from_pages(pages):
    full = build_full_text_with_page_markers(pages)

    # Pattern: find blocks that start with a number and dot (e.g., "1.", "23.")
    # DOTALL so '.' matches newlines. We keep the block from one number to the next number or EOF.
    pattern = re.compile(r'(?s)(?:^|\n)\s*(\d{1,3}\.)\s*(.*?)(?=(?:\n\s*\d{1,3}\.)|\Z)')
    chunks = []

    for m in pattern.finditer(full):
        num = m.group(1).strip()          # e.g., "1."
        block = m.group(2).strip() or ""  # the text after the number until next number

        # Find which pages this block spans by searching for page markers inside the match span
        block_span = full[m.start():m.end()]
        pages_found = re.findall(r'<<PAGE_(\d+)>>', block_span)
        if pages_found:
            page_start = int(pages_found[0])
            page_end = int(pages_found[-1])
        else:
            page_start = page_end = None

        # Clean block: remove inline page markers to simplify splitting
        block_clean = re.sub(r'<<PAGE_\d+>>', ' ', block)

        # Heuristic to split question vs answer:
        # 1) If '**' appears (your FAQ uses **), split at first '**' occurrence
        # 2) Else, if there's a '?' within the first 200 chars, split at first '?'
        # 3) Else split at first newline (reasonable fallback)
        q_text = ""
        a_text = ""
        if '**' in block_clean:
            # split on the first occurrence of '**' (or multiple)
            parts = block_clean.split('**', 1)
            # If the pattern is "Q ... **  - Answer..." often the question is before **
            # But sometimes ** marks the question or the "answer start", so we'll test lengths
            if len(parts[0].strip()) < 400:  # treat left part as question if not huge
                q_text = parts[0].strip()
                a_text = parts[1].strip()
            else:
                # fallback: take first sentence as question
                first_q = re.split(r'(?<=[\?\.\!])\s', block_clean, maxsplit=1)
                q_text = first_q[0].strip()
                a_text = first_q[1].strip() if len(first_q) > 1 else ""
        else:
            # find first '?' in the first N chars
            head = block_clean[:600]  # look in first 600 chars for a question mark
            qm = head.find('?')
            if qm != -1:
                # split at the first question mark (include it)
                q_text = block_clean[:qm+1].strip()
                a_text = block_clean[qm+1:].strip()
            else:
                # fallback: split at the first newline that seems to separate the answer
                parts = re.split(r'\n\s*\n', block_clean, maxsplit=1)  # blank-line separator
                if len(parts) > 1:
                    q_text = parts[0].strip()
                    a_text = parts[1].strip()
                else:
                    # very last fallback: try first sentence
                    first_sentence = re.split(r'(?<=[\.\!\?])\s', block_clean, maxsplit=1)
                    q_text = first_sentence[0].strip()
                    a_text = first_sentence[1].strip() if len(first_sentence) > 1 else ""

        # Final cleanup: remove excessive whitespace
        q_text = re.sub(r'\s+', ' ', (q_text or "")).strip()
        a_text = re.sub(r'\s+', ' ', (a_text or "")).strip()

        # If the "question" we detected is empty, try to reconstruct as "Question <num>"
        if not q_text:
            q_text = f"{num} (Question text not detected)"
        # If answer empty, set explicit note (so retrieval can show it's incomplete)
        if not a_text:
            a_text = "(answer not found in block - may be on adjacent pages)"

        chunk = {
            "id": str(uuid.uuid4()),
            "number": num,
            "page_start": page_start,
            "page_end": page_end,
            "question": q_text,
            "answer": a_text,
            "text": f"Question: {q_text}\nAnswer: {a_text}"
        }
        chunks.append(chunk)

    return chunks

# Run the chunker on your `pages` (you already have pages from PdfReader)
qa_chunks = qa_chunks_from_pages(pages)
print("Detected Q&A chunks:", len(qa_chunks))
# show first few nicely
for c in qa_chunks[:6]:
    print("\n---")
    print("ID:", c["id"])
    print("Num:", c["number"], "Pages:", c["page_start"], "-", c["page_end"])
    print("Q:", c["question"])
    print("A:", c["answer"])

Detected Q&A chunks: 45

---
ID: e6d891b8-d003-407e-9187-246f9e77aba7
Num: 1. Pages: None - None
Q: Are you guy’s insurance company or Broker?
A: Not really, we are neither an insurance company nor a broker. Basically, we are an independent price searching company, search on your behalf with your consent to find you reasonable online prices. We do not work as a broker or agent for any specific company but a Non-Regulated Introducer for 'Kindertons Limited' (FRN 306969). and 'Worldin Holding Limited' who are authorised and regulated by Financial Conduct Authority (FCA). However, we are registered with the Information Commissioner's Office (ICO) and all your information are protected under the 'Data Protection Act & General Data Protection Regulation (GDPR 2018). We can assist the valuable customers with very helpful service as our multilingual dedicated friendly staffs are waiting to serve. Also, you are strongly advised to read our Terms and Conditions and Privacy Policy thoroughly bef

In [3]:
import sys
!{sys.executable} -m pip install sentence-transformers

In [2]:
# Cell — build FAISS index from qa_chunks
import faiss, pickle
from sentence_transformers import SentenceTransformer
import numpy as np

# Make sure qa_chunks exists from your previous code
assert "qa_chunks" in globals(), "qa_chunks not found — run your chunking code first."

EMB_MODEL = "all-MiniLM-L6-v2"  # lightweight & fast model
embedder = SentenceTransformer(EMB_MODEL)

# Extract texts (Q + A together) for embeddings
texts = [c["text"] for c in qa_chunks]
embeddings = embedder.encode(texts, convert_to_numpy=True).astype("float32")

# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

# Save index
faiss.write_index(index, "qa_book_faiss.index")

# Save meta (ids, page ranges, Q/A)
metadatas = [{"id": c["id"], "question": c["question"], "answer": c["answer"],
              "page_start": c["page_start"], "page_end": c["page_end"]} for c in qa_chunks]
with open("qa_book_meta.pkl", "wb") as f:
    pickle.dump({"texts": texts, "metadatas": metadatas}, f)

print(f"FAISS index built with {index.ntotal} vectors and saved.")

FAISS index built with 45 vectors and saved.


In [3]:
# Cell 1 — load index & metadata
import faiss, pickle, os
from sentence_transformers import SentenceTransformer

FAISS_INDEX_FILE = "qa_book_faiss.index"
META_FILE = "qa_book_meta.pkl"
EMB_MODEL = "all-MiniLM-L6-v2"

assert os.path.exists(FAISS_INDEX_FILE), "FAISS index not found. Run the build cell first."
assert os.path.exists(META_FILE), "Meta file not found. Run the build cell first."

index = faiss.read_index(FAISS_INDEX_FILE)
with open(META_FILE, "rb") as f:
    meta = pickle.load(f)

texts = meta["texts"]
metadatas = meta["metadatas"]

embedder = SentenceTransformer(EMB_MODEL)

print("Loaded FAISS index with vectors:", index.ntotal)
print("Loaded meta entries:", len(metadatas))
# show top 1 example
print("\nExample metadata[0]:")
print(metadatas[0])
print("\nExample text excerpt:")
print(texts[0].replace("\n"," "))

Loaded FAISS index with vectors: 45
Loaded meta entries: 45

Example metadata[0]:
{'id': 'e6d891b8-d003-407e-9187-246f9e77aba7', 'question': 'Are you guy’s insurance company or Broker?', 'answer': "Not really, we are neither an insurance company nor a broker. Basically, we are an independent price searching company, search on your behalf with your consent to find you reasonable online prices. We do not work as a broker or agent for any specific company but a Non-Regulated Introducer for 'Kindertons Limited' (FRN 306969). and 'Worldin Holding Limited' who are authorised and regulated by Financial Conduct Authority (FCA). However, we are registered with the Information Commissioner's Office (ICO) and all your information are protected under the 'Data Protection Act & General Data Protection Regulation (GDPR 2018). We can assist the valuable customers with very helpful service as our multilingual dedicated friendly staffs are waiting to serve. Also, you are strongly advised to read our Te

In [25]:
# Cell 2 — retrieval utility
import numpy as np

def retrieve(question, k=4):
    # create embedding (float32 for faiss)
    q_emb = embedder.encode([question], convert_to_numpy=True).astype("float32")
    D, I = index.search(q_emb, k)
    results = []
    for dist, idx in zip(D[0], I[0]):
        results.append({
            "score": float(dist),
            "text": texts[idx],
            "meta": metadatas[idx]
        })
    return results

# Quick interactive test (edit question as needed)
q = "what are the conditions for a provisional drivers insurance??"
res = retrieve(q, k=5)
for i, r in enumerate(res, 1):
    m = r["meta"]
    print(f"\n--- Result {i} (score={r['score']:.4f}) pages {m['page_start']}-{m['page_end']} ---")
    print("Q:", m["question"])
    print("A excerpt:", m["answer"][:400].replace("\n"," "))


--- Result 1 (score=0.8083) pages None-None ---
Q: Will a provisional driving licence holder be able to buy insurance?
A excerpt: * - Yes sure they can but the UK driving law requires them to drive under supervision. A U k manual driver must be with him who is aged over 21 and license is 3 years old also an ‘L Plate/sign’ to be displayed at the front and back of the car.

--- Result 2 (score=1.0004) pages None-None ---
Q: What is a provisional Licence UK?
A excerpt: ** You must have a provisional driving licence for Great Britain or Northern Ireland when you're learning to drive or ride. You must be supervised when you're learning to drive a car. This can be by a driving instructor or someone else who meets the rules, for example family or friends.

--- Result 3 (score=1.0708) pages None-None ---
Q: Will I be able to drive another car?
A excerpt: * - Well’ it depends on the insurance companies mainly, if the driver meets certain rules which is --- - The driver Has to be a Policy Holde

In [4]:
# Cell 3 — Ollama synthesis (optional)
import requests, textwrap, json

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.2:3b"   # change if you use a different small model
TIMEOUT = 60

def synthesize_with_ollama(question, retrieved, max_tokens=256):
    # build context from retrieved meta (question + answer)
    context = "\n\n---\n\n".join([
        f"(pages {r['meta']['page_start']}-{r['meta']['page_end']}) Q: {r['meta']['question']}\nA: {textwrap.shorten(r['meta']['answer'], 900, placeholder=' ...')}"
        for r in retrieved
    ])
    prompt = f"""You are an assistant that MUST answer using ONLY the provided passages. If the answer is not in the passages, reply: "I don't know from the provided text."

Passages:
{context}

Question: {question}

Answer concisely and cite which page range(s) you used.
"""
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": 0.0
    }
    resp = requests.post(OLLAMA_URL, json=payload, timeout=TIMEOUT)
    resp.raise_for_status()
    data = resp.json()
    # Try common response keys (adjust if your Ollama returns different)
    if isinstance(data, dict):
        return data.get("response") or data.get("output") or json.dumps(data, indent=2)
    return str(data)

# Example usage:
question = "Are you an insurance company or broker?"
retrieved = retrieve(question, k=4)
print("\nRetrieved passages titles:")
for r in retrieved:
    print("-", r["meta"]["question"], f"(pages {r['meta']['page_start']}-{r['meta']['page_end']})")
print("\nRequesting Ollama to synthesize...\n")
try:
    print(synthesize_with_ollama(question, retrieved))
except Exception as e:
    print("Ollama request failed:", e)
    # optionally print raw response if available

NameError: name 'retrieve' is not defined

In [8]:
import requests, json, textwrap
OLLAMA_URL = "http://localhost:11434/api/generate"
payload = {"model": "llama3.2:3b", "prompt": "hello", "max_tokens": 16}

try:
    r = requests.post(OLLAMA_URL, json=payload, timeout=30)
    print("HTTP", r.status_code)
    print("----- resp.text preview -----")
    print(r.text[:4000])   # print first 4000 chars
    print("----- end preview -----")
except Exception as e:
    print("Request failed:", repr(e))

HTTP 200
----- resp.text preview -----
{"model":"llama3.2:3b","created_at":"2025-09-22T09:37:24.0278575Z","response":"Hello","done":false}
{"model":"llama3.2:3b","created_at":"2025-09-22T09:37:24.1931391Z","response":"!","done":false}
{"model":"llama3.2:3b","created_at":"2025-09-22T09:37:24.3504356Z","response":" It","done":false}
{"model":"llama3.2:3b","created_at":"2025-09-22T09:37:24.5105306Z","response":"'s","done":false}
{"model":"llama3.2:3b","created_at":"2025-09-22T09:37:24.6691414Z","response":" nice","done":false}
{"model":"llama3.2:3b","created_at":"2025-09-22T09:37:24.8287099Z","response":" to","done":false}
{"model":"llama3.2:3b","created_at":"2025-09-22T09:37:24.9970772Z","response":" meet","done":false}
{"model":"llama3.2:3b","created_at":"2025-09-22T09:37:25.1580679Z","response":" you","done":false}
{"model":"llama3.2:3b","created_at":"2025-09-22T09:37:25.3202097Z","response":".","done":false}
{"model":"llama3.2:3b","created_at":"2025-09-22T09:37:25.4896304Z","response"

In [6]:
import numpy as np
import faiss, pickle

# Load FAISS + meta if not already loaded
FAISS_INDEX_FILE = "qa_book_faiss.index"
META_FILE = "qa_book_meta.pkl"

if "index" not in globals():
    index = faiss.read_index(FAISS_INDEX_FILE)
if "meta" not in globals():
    with open(META_FILE, "rb") as f:
        meta = pickle.load(f)

texts = meta["texts"]
metadatas = meta["metadatas"]

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def retrieve(query, k=4):
    """Search FAISS index and return top-k matches with their metadata."""
    query_emb = embedder.encode([query], convert_to_numpy=True).astype("float32")
    D, I = index.search(query_emb, k)
    results = []
    for i, idx in enumerate(I[0]):
        if idx < 0 or idx >= len(texts):
            continue
        results.append({
            "score": float(D[0][i]),
            "text": texts[idx],
            "meta": metadatas[idx]
        })
    return results

# Quick test
print("Test retrieval results:")
for r in retrieve("insurance company or broker?", k=2):
    print("-", r["meta"]["question"], "(pages", r["meta"]["page_start"], "-", r["meta"]["page_end"], ")")

Test retrieval results:
- Are you guy’s insurance company or Broker? (pages None - None )
- Do you do taxi Insurance? (pages None - None )


In [10]:
# Ollama NDJSON-safe synth (paste into a Jupyter cell)
import requests, json, textwrap

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.2:3b"
TIMEOUT = 180  # allow cold-starts

def ollama_generate_concat(prompt, model=MODEL_NAME, timeout=TIMEOUT, max_tokens=256):
    payload = {"model": model, "prompt": prompt, "max_tokens": max_tokens, "temperature": 0.0}
    try:
        resp = requests.post(OLLAMA_URL, json=payload, timeout=timeout)
        resp.raise_for_status()
        text = resp.text
        # Split by newlines and parse JSON per-line (NDJSON)
        lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
        pieces = []
        last_obj = None
        for ln in lines:
            try:
                obj = json.loads(ln)
            except Exception:
                # try to strip "data:" prefix if present
                if ln.startswith("data:"):
                    try:
                        obj = json.loads(ln[len("data:"):].strip())
                    except Exception:
                        obj = None
                else:
                    obj = None
            if obj is None:
                continue
            # some responses use "response" field per chunk
            part = obj.get("response") or obj.get("text") or obj.get("output")
            if isinstance(part, dict):
                # if nested, stringify it
                part = json.dumps(part)
            if part:
                pieces.append(str(part))
            last_obj = obj
        # join pieces with no extra spaces (they are already token chunks)
        joined = "".join(pieces).strip()
        # if joined empty, attempt other fallbacks
        if not joined and last_obj:
            # try common keys in the last obj
            for k in ("response", "output", "result", "text"):
                if k in last_obj:
                    joined = str(last_obj[k])
                    break
        return joined if joined else None
    except Exception as e:
        print("Ollama request failed:", repr(e))
        return None

def synthesize_with_ollama_streaming_or_fallback(question, retrieved, max_tokens=256):
    context = "\n\n---\n\n".join([
        f"(pages {r['meta']['page_start']}-{r['meta']['page_end']}) Q: {r['meta']['question']}\nA: {textwrap.shorten(r['meta']['answer'], 900, placeholder=' ...')}"
        for r in retrieved
    ])
    prompt = f"""You are an assistant that MUST answer using ONLY the provided passages. If the answer is not in the passages, reply: "I don't know from the provided text."

Passages:
{context}

Question: {question}

Answer concisely and cite which page range(s) you used.
"""
    out = ollama_generate_concat(prompt, max_tokens=max_tokens)
    if out:
        return out
    # fallback: return top retrieved answers
    best = "\n\n".join([f"Page {r['meta']['page_start']}-{r['meta']['page_end']}: {r['meta']['answer']}" for r in retrieved[:3]])
    return f"(Fallback — top retrieved answers)\n\n{best}"

# Example prompt:
q = "?"
retrieved = retrieve(q, k=4)   
print(synthesize_with_ollama_streaming_or_fallback(q, retrieved))

The person involved in the accident, along with their personal belongings' value, would be responsible for paying. However, "if fault accident- please give him his insurer phone number" suggests that if it's a fault accident, the insurance company will likely cover the costs. 

Since this question is answered on pages 7-7 (comprehensive cover), and also pages None-None (third party, fire and theft cover) mentions damage to other vehicles or people but not payment for damages, I don't know from the provided text.

However, in page 3-3, when it's a non-fault accident, "take his involved car registration number, involved driver name of his car at the time of accident, the place & time of accident, his right contact number then simply pass it over to the Kindertons for the rest and advise the client not to contact insurer" implies that someone else will pay in case of non-fault.

To answer more concisely: If non-fault, "Kindertons" would likely cover costs.
